In [2]:

!pip install datasets
!pip install transformers
!pip install sacremoses 
!pip install evaluate
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.6 MB/s 


In [3]:
import numpy as np
from datasets import load_dataset, Dataset
import evaluate

from transformers import DataCollatorForTokenClassification, AutoModelForSequenceClassification, TrainingArguments, \
    Trainer, AutoTokenizer, DataCollatorWithPadding

In [4]:
dataset1 = load_dataset("poleval2019_cyberbullying", "task01")
dataset2 = load_dataset("poleval2019_cyberbullying", "task02")

Generating train split:   0%|          | 0/10041 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset poleval2019_cyberbullying downloaded and prepared to /root/.cache/huggingface/datasets/poleval2019_cyberbullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10041 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset poleval2019_cyberbullying downloaded and prepared to /root/.cache/huggingface/datasets/poleval2019_cyberbullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load("accuracy")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
def transformer_classifier(dataset: Dataset, name: str, num_labels: int):
    tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True)

    tokenized_data = dataset.map(preprocess_function, batched=True)
    model = AutoModelForSequenceClassification.from_pretrained('allegro/herbert-base-cased',
                                                               num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir=f'/home/var_{num_labels}',
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data['train'],
        eval_dataset=tokenized_data['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()
    model.save_pretrained(f'/content/drive/MyDrive/colab_data/NLP/mod_{name}')
    return model

In [15]:
mod1 = transformer_classifier(dataset1, "roberta-detection-bullying-1", 2)
print("done training one")
mod2 = transformer_classifier(dataset2, "roberta-detection-bullying-2", 3)
print("done training two")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

Epoch,Training Loss,Validation Loss,Accuracy
1,0.246600,0.357271,0.892000
2,0.187000,0.449836,0.896000
3,0.148400,0.387407,0.903000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to /home/var_2/checkpoint-628
Configuration saved in /home/var_2/checkpoint-628/config.json
Model weights saved in /home/var_2/checkpoint-628/pytorch_model.bin
tokenizer config file saved in /home/var_2/checkpoint-628/tokenizer_config.json
Special tokens file saved in /home/var_2/checkpoint-628/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1

done training one


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.292000,0.414834,0.867000
2,0.216800,0.465595,0.889000
3,0.167200,0.459510,0.892000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to /home/var_3/checkpoint-628
Configuration saved in /home/var_3/checkpoint-628/config.json
Model weights saved in /home/var_3/checkpoint-628/pytorch_model.bin
tokenizer config file saved in /home/var_3/checkpoint-628/tokenizer_config.json
Special tokens file saved in /home/var_3/checkpoint-628/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1

done training two


In [27]:
import torch
from transformers import pipeline

text = "@anonymized_account @anonymized_account skąd wiesz jaki Skendija ma budżet skoro mówisz że jest bogatsza ? Tylko dwóch zawodników ponoć dobrze zarabia."

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
device = torch.device("cuda:0")
tokenized_text = tokenizer(text, return_tensors="pt").to(device)



with torch.no_grad():
  logits = mod1(**tokenized_text).logits
logits

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

tensor([[ 2.7995, -2.5159]], device='cuda:0')

In [32]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

def evaluate_task1(predicted, actual):
    return {"accuracy": accuracy_score(predicted, actual), "precision": precision_score(predicted, actual), "recall": recall_score(predicted, actual), "f1": f1_score(predicted, actual)} 

def evaluate_task2(predicted, actual):
    return {"accuracy": accuracy_score(predicted, actual), "precision": precision_score(predicted, actual, average='macro'), "recall": recall_score(predicted, actual, average='macro'), "f1": f1_score(predicted, actual, average='macro')} 
   

In [28]:
def compute_metrics_singleclass(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return evaluate_task1(pred, labels)

def compute_metrics_multiclass(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return evaluate_task2(pred, labels)

def evaluate_transformers(model, dataset, tokenizer_name, compute_metrics):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenized_dt = dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True)
    trainer = Trainer(model=model,
                      eval_dataset=tokenized_dt["test"],
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics)

    return trainer.evaluate()

def transformers_to_scores(eval_res):
    return {"accuracy": eval_res["eval_accuracy"], "precision": eval_res["eval_precision"], "recall": eval_res["eval_recall"], "f1": eval_res["eval_f1"]} 

In [33]:

score1 = transformers_to_scores(
    evaluate_transformers(mod1, dataset1, "allegro/herbert-base-cased", compute_metrics_multiclass))
score2 = transformers_to_scores(
    evaluate_transformers(mod2, dataset2, "allegro/herbert-base-cased", compute_metrics_multiclass))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
You're using a HerbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
score1

{'accuracy': 0.892,
 'precision': 0.6096308296852918,
 'recall': 0.8873462428449641,
 'f1': 0.6490954460386775}

In [35]:
score2

{'accuracy': 0.867,
 'precision': 0.3363914373088685,
 'recall': 0.6222889556222889,
 'f1': 0.31562271508652207}